# Analysis of human behavior on single scenario

**The purpose of this notebook is to:** 
* Apply preprocessing to human behavioral data
* Visualize distribution and compute summary statistics over **human** physical judgments
* Output CSV that can be re-loaded into R notebook for statistical modeling & fancy visualizations

**This notebook depends on:**
* Running `./generate_dataframes.py` (INTERNAL USE ONLY)
* Running `./upload_results.py` (INTERNAL USE ONLY)
* Running `./download_results.py` (PUBLIC USE)

## setup

#### Load packages

In [ ]:
import os
import sys
import urllib, io

sys.path.append('./analysis_helpers')
from importlib import reload
from analysis_helpers import *

import numpy as np
import scipy.stats as stats
import pandas as pd

import pymongo as pm
from collections import Counter
import json
import re
import ast

from PIL import Image, ImageOps, ImageDraw, ImageFont 

from io import BytesIO
import base64

from tqdm.notebook import tqdm

import  matplotlib
from matplotlib import pylab, mlab, pyplot
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
plt.style.use('seaborn-white')

import seaborn as sns

%matplotlib inline
import scipy.stats
import random

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

#### options

In [ ]:
# display all columns
pd.set_option('display.max_columns', None)

# seaborn plotting themes
sns.set_context('talk')
sns.set_style("whitegrid")

#### set up paths and directories

In [ ]:
## directory & file hierarchy
proj_dir = os.path.abspath('..')
datavol_dir = os.path.join(proj_dir,'data')
analysis_dir =  os.path.abspath('.')
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
json_dir = os.path.join(results_dir,'json')
exp_dir = os.path.abspath(os.path.join(proj_dir,'behavioral_experiments'))
png_dir = os.path.abspath(os.path.join(datavol_dir,'png'))

## add helpers to python path
if os.path.join(proj_dir,'stimuli') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'stimuli'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       
    
## add helpers to python path
if os.path.join(analysis_dir,'utils') not in sys.path:
    sys.path.append(os.path.join(analysis_dir,'utils'))   

def make_dir_if_not_exists(dir_name):   
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
    return dir_name

## create directories that don't already exist        
result = [make_dir_if_not_exists(x) for x in [results_dir,plot_dir,csv_dir]]

### load in data

In [ ]:
from experiment_meta import *
HEM = pd.DataFrame(NEURIPS2021_EXPS) # HEM = "human experiment metadata"
HEM

In [ ]:
## get paths to all human response data
data_paths = [os.path.join(csv_dir,'humans',i) for i in os.listdir(os.path.join(csv_dir,'humans'))]
resp_paths = [i for i in data_paths if i.split('/')[-1].split('-')[0]=='human_responses']
assert len(resp_paths)==8

In [ ]:
## load in example dataframe
exp_ind = 0
d = pd.read_csv(resp_paths[exp_ind])
scenarioName = resp_paths[exp_ind].split('/')[-1].split('-')[1].split('_')[0]

## some utility vars
colnames_with_variable_entries = [col for col in sorted(d.columns) if len(np.unique(d[col]))>1]
colnames = ['gameID','trialNum','prolificIDAnon','stim_ID','response','target_hit_zone_label','correct','choices','rt']
# colnames = ['gameID','trialNum','stim_ID','response','target_hit_zone_label','correct','choices','rt']

## subset dataframe by colnames of interest
_D = d[colnames]
_D = _D.assign(scenarioName = scenarioName)

## preprocess RTs (subtract 2500ms presentation time, log transform)
_D = _D.assign(RT = _D['rt'] - 2500) 
_D = _D.assign(logRT = np.log(_D['RT']))
_D = _D.drop(columns=['rt'],axis=1)

## convert responses to boolean
binary_mapper = {'YES':True, 'NO':False}
_D = _D.assign(responseBool = _D['response'].apply(lambda x: binary_mapper[x]), axis=0)

print('Currently analyzing the {} experiment.'.format(scenarioName))

### Data exclusion criteria (from `preregistration_neurips2021.md`)

 Data from an entire experimental session will be excluded if the responses:
 * contain a sequence with unusually long streak, defined as occurring less than 2.5% of the time under random responding
 * contain a sequence of at least 24 trials alternating "yes" and "no" responses
 * are correct for fewer than 4 out of 10 familiarization trials (i.e., 30% correct or lower)
 * the mean accuracy for that participant is below 3 standard deviations below the median accuracy across all participants \for that scenario
 * the mean log-transformed response time for that participant is 3 standard deviations above the median log-transformed response time across all participants for that scenario
 
Excluded sessions will be flagged. Flagged sessions will not be included in the main analyses. We will also conduct our planned analyses with the flagged sessions included to investigate the extent to which the outcomes of the main analyses change when these sessions are included. Specifically, we will fit a statistical model to all sessions and estimate the effect of a session being flagged on accuracy. 

In [ ]:
from analysis_helpers import *
D = apply_exclusion_criteria(_D)

## Visualize distribution and compute summary statistics over human physical judgments



### Human accuracy across participants for each stimulus
We will analyze accuracy for each stimulus by computing the proportion of correct responses across all participants who viewed that stimulus. 



In [ ]:
Dacc = D.groupby('prolificIDAnon').agg({'correct':np.mean})
h = sns.histplot(data=Dacc, x='correct', bins=30, stat='probability')
t = plt.title('Accuracy distribution across participants')

In [ ]:
def bootstrap_mean(D, col = 'correct', nIter=1000):
    bootmean = []
    for currIter in np.arange(nIter):
        bootD = D.sample(n=len(D),random_state=currIter, replace=True)            
        bootmean.append(np.mean(bootD[col].values))
    return bootmean

In [ ]:
bootmeans = bootstrap_mean(Dacc, col='correct', nIter=1000)
lb = np.percentile(bootmeans,2.5)
ub = np.percentile(bootmeans,97.5)
print('The observed mean accuracy for {} is {}.'.format(scenarioName, np.round(Dacc['correct'].mean(),3)))
print('95% CI: [{},{}].'.format(lb.round(3),ub.round(3)))

### Human accuracy across stimuli for each participant
We will analyze accuracy for each participant by computing the proportion of correct responses across all stimuli. 



In [ ]:
Dacc = D.groupby('stim_ID').agg({'correct':np.mean})
h = sns.histplot(data=Dacc, x='correct', bins=30, stat='probability')
t = plt.title('Accuracy distribution across stimuli')

### Human-human consistency for each stimulus
We will estimate human-human consistency for each stimulus by computing the proportion of responses that match the modal response for that stimulus (whether that modal response is correct or incorrect).



In [ ]:
assert len(np.unique(D.groupby('stim_ID').agg({'correct':'count'})['correct'].values))==1 # sanity check
numSubs = np.unique(D.groupby('stim_ID').agg({'correct':'count'})['correct'].values)[0] # get num subs
Dmode = D.groupby('stim_ID').agg({'responseBool':scipy.stats.mode}) # compute modal response
propModalResponse = Dmode['responseBool'].apply(lambda x: x[1][0]).values / numSubs # get proportion modal response
Dmode = Dmode.assign(propModalResponse = Dmode['responseBool'].apply(lambda x: x[1][0]).values / numSubs) 
Dcombined = Dacc.merge(Dmode, left_index=True, right_index=True)

In [ ]:
h = sns.histplot(data=Dmode, x='propModalResponse', bins=30, stat='probability')
t = plt.title('Distribution of agreement among participants (similar to above, but not exactly the same)')

### Human-human consistency across stimuli (within scenario)
We will analyze human-human consistency by computing the mean correlation between (binary) response vectors produced by each human participant across all stimuli within each scenario. 



In [ ]:
## create response feature matrix (numSubs x numTrialsPerSub)
D2 = D.sort_values(by=['prolificIDAnon','stim_ID']).reset_index(drop=True)
numSubs = len(np.unique(D['prolificIDAnon'].values))
numTrialsPerSub = int(len(D)/numSubs)
respMat = np.reshape(D2['responseBool'].values, (numSubs,numTrialsPerSub)) 

## sanity check that the reshape operation happened correctly
assert len([i for (i,j) in list(zip(respMat[0],D2[:150]['responseBool'].values)) if i!=j])==0

In [ ]:
## get pairwise correlation distances
dists = scipy.spatial.distance.pdist(respMat, metric='correlation')
corrMat = scipy.spatial.distance.squareform(dists)

In [ ]:
## distribution of pairwise correlations between participant response vectors
h = sns.histplot(corrMat[np.triu_indices(n=len(corrMat), k=1)])
t = plt.title('pairwise correlations between participant response vectors')

In [ ]:
pairwiseCorrs = corrMat[np.triu_indices(n=len(corrMat), k=1)]
lb = np.percentile(pairwiseCorrs, 2.5)
ub = np.percentile(pairwiseCorrs, 97.5)
print('The median pairwise correlation is {}.'.format(np.percentile(pairwiseCorrs, 50).round(3)))
print('95% CI: [{},{}].'.format(lb.round(3),ub.round(3)))

In [ ]:
## plot heatmap of correlation distances between participants
reallyRun = False
if reallyRun:
    sns.heatmap(corrMat, square=True)

### Human accuracy as a function of stimulus attributes
We will conduct exploratory analyses of human accuracy as a function of various scenario-specific stimulus attributes that varied across trials. We will examine those stimulus attributes that varied across stimuli within each scenario and explore the relationship between each individual attribute and human accuracy, as well as beetween linear combinations of them and human accuracy. 



In [ ]:
## TODO

### Human accuracy by scenario
We will fit human responses across all scenarios with a mixed-effects logistic regression model, including scenario as a fixed effect and participants and individual stimuli as random effects.

In [ ]:
D

In [ ]:
from pymer4.models import Lmer

In [ ]:
model = Lmer('correct ~ (1|prolificIDAnon) + (1|stim_ID)',data = D, family='binomial')

In [ ]:
model.fit()

# Other exploratory human behavioral analyses
* We will explore the relation of demographic variables on the performance of participants: how does age, gender, and educational status relate to the overall accuracy of a subject?
* We will additionally explore any potential left/right or yes/no response biases. 
